# [Django Models](https://docs.djangoproject.com/en/dev/topics/db/models/)

## Creating New Models

After creating a new app, you'll probably start defining that app's models. Apps need to be registered to `settings.INSTALLED_APPS` so the project is aware of it, making `makemigrations` and `migrate` commands work.

Django models are Python classes that inherit from `django.db.models.Model`. Attributes of this class represent database rows. Many kinds of fields are available to represent different data types and relationships.

Using the management commands, create a new project called "google" and an app for "account". Add the model below.

```
from django.db import models

class Profile(models.Model):
    name = models.CharField(max_length=42)
```

Use the `python manage.py makemigrations` command. If you haven't added your app to the `INSTALLED_APPS`, no changes will be detected.

Register it to `INSTALLED_APPS` and use the `makemigrations` command. If it succeeds, the files created will be shown. Now you can use the `python manage.py migrate` command. Whenever we update a model, use the same process to create migration scripts and apply them to the database.

## Modifying Existing Models

After the model has been applied to the database, we can change our mind. It happens. I decided to add new attributes to the data. Add the two new fields below.

```
from django.db import models

class Profile(models.Model):
    name = models.CharField(max_length=42)
    photo = models.ImageField()
    created = models.DateField()
```

Use `makemigrations`. Oh no, another error!

```
You are trying to add a non-nullable field 'photo' to profile without a default; we can't do that (the database needs something to populate existing rows).
Please select a fix:
 1) Provide a one-off default now (will be set on all existing rows)
 2) Quit, and let me add a default in models.py
Select an option:
```

The problem is exactly what the error says. We are adding new rows and the database needs to populate it. We need to provide defaults.

## [Field Options](https://docs.djangoproject.com/en/dev/ref/models/fields/#field-options)

When adding new fields, a default is required. This is because new rows will be added to the database and 1) it requires a value, and 2) it doesn't know what value to put in it. Let's null a field and add a default. Check the [documentation](https://docs.djangoproject.com/en/dev/ref/models/fields/) for details on [null](https://docs.djangoproject.com/en/dev/ref/models/fields/#null) and [blank](https://docs.djangoproject.com/en/dev/ref/models/fields/#blank) (they usually go together), and [default](https://docs.djangoproject.com/en/dev/ref/models/fields/#default). 

```
from datetime import datetime
from django.db import models

class Profile(models.Model):
    name = models.CharField(max_length=42)
    photo = models.ImageField(null=True, blank=True)
    created = models.DateField(default=datetime.now)
```

There are many more arguments available for fields but most of them are optional. Certain fields require certain field options but these are only a few. We won't get into the details in our basic covereage. The [documentation](https://docs.djangoproject.com/en/dev/ref/models/fields/) provides the more details about each one.

## [Field Types](https://docs.djangoproject.com/en/dev/ref/models/fields/#field-types)

We demonstrated a few fields in our model. the `CharField`, `Imagefield`, and `DateField`. There are many more fields available for different data types.

Django's default behavior is to give each model an auto-incrementing `id` field. This is an example of [`AutoField`](https://docs.djangoproject.com/en/dev/ref/models/fields/#autofield).

Checkboxes are examples of [`BooleanField`](https://docs.djangoproject.com/en/dev/ref/models/fields/#booleanfield).

At first, it may take a little research to find the right field type for your data. But you'll easily get familiar with the common ones as you use them. If there's a specific data type that's not covered by built-in fields, or you want a custom implementation, there are always third-party packages or you can create your own custom field. The [documentation](https://docs.djangoproject.com/en/dev/ref/models/fields/) provides the more details about each one.

## [Admin](https://docs.djangoproject.com/en/dev/ref/contrib/admin/)

We now have a working model with some basic profile information. Let's register it to the [admin](https://docs.djangoproject.com/en/dev/ref/contrib/admin/) and see how it looks.

```
from django.contrib import admin

from .models import Profile

admin.site.register(Profile)
```

If you haven't registered a superuser yet, we need one now so you can login to `admin/`. We just plainly registered our model to the admin. It can in fact be customized in many ways. You'll see how to do that in the documentation and other tutorials. Just registering the model allows us to interact with it already, without making any views or controllers. This speeds up feedback and we can fix things faster. 

## [Using Models](https://docs.djangoproject.com/en/dev/ref/models/)

### [QuerySet](https://docs.djangoproject.com/en/dev/ref/models/querysets/#django.db.models.query.QuerySet.order_by)

Open a terminal window and run the `python manage.py shell` command. We'll use our model from the shell just so we can see what the code is like and see the results right away. Code that works here can be easily rewritten in `views.py`.

Let's import our class and retrieve data from the database.

```
from account.models import Profile

Profile.objects.all()
```

This returns a `QuerySet` containing all instances of our class in our database table. Usually we want just a small part of the data so we don't want to slow shings down by querying all the data then filtering it again.

### Creating and Editing

We haven't created any objects yet so this returns an empty `QuerySet`. Let's create an intance, then save it to the database. You can use the `admin/` to do this. You can also do it in your code.

```
profile = Profile(name="Guido Van Rossum")
profile.save()
```

You need to supply the minimum required fields for it to work. In our case, `Profile` instances only need a name because the `photo` and `created` fields can be `null/blank` or have a `default`, respectively.

Let's try to retrieve objects again and see what happens.

```
Profile.objects.all()
```

Now we know how to create and save objects into the database, let's edit and update the information.

We still have our `profile` object (name="Guido Van Rossum"). I remember he's Dutch and the Dutch "Van" preposition is not capitalized. Let's fix that.

```
profile.name = "Guido van Rossum"  # the Dutch van is not capitalized
profile.save()
Profile.objects.all()
```

That updates our info in the database. Django knows if it's the same instance. If you change any of its attributes and call `.save()` on it, the database is updated.

### String Representation of Objects

`[<Profile: Profile object>]` wasn't very helpful. We know what class it belongs to but `Profile object` isn't very descriptive. Let's change that by adding a `__str__` method to our object.

```
class Profile(models.Model):
    name = models.CharField(max_length=42)
    photo = models.ImageField(null=True, blank=True)
    created = models.DateField(default=datetime.now)

    def __str__(self):
        return self.name
```

The local server should detect changes and restart. Now let's try that again.

```
Profile.objects.all()
```

Now that's better. `[<Profile: Guido van Rossum>]` tells us which instance from what class. Overriding `__str__` makes a big difference on readability.

### Primary Keys

Let's get an object from our `QuerySet`. This time We'll do it by slicing the result.

```
profile = Profile.objects.all()[0]
profile.pk
```

Django provides a `pk` or “primary key” shortcut. By default, it refers to `id`.

```
profile.pk is profile.id
```

Django gives each model an auto-incrementing `id` field. This behavior can be overridden by specifying exactly one field with the field option [`primary_key=True`](https://docs.djangoproject.com/en/dev/ref/models/fields/#django.db.models.Field.primary_key).

### Filtering and Excluding

We used `.all()` to retrieve all our data. Now we'll use the `.filter()` and `.exclude()` methods to retrieve a limited set.

```
person = Person.objects.filter(id=1)
```

#### Field Lookups

Database queries can accept keyword arguments like our first example. It can also accept [field lookups](https://docs.djangoproject.com/en/dev/ref/models/querysets/#field-lookups) equivalent to the SQL `WHERE` clause. 
```
from datetime import datetime

# datetime.now().year = the current year
created_this_year = Person.objects.filter(created__year=datetime.now().year)
not_created_this_year = Person.objects.exclude(created__year=datetime.now().year)  # we don't have any
```

Better check the documentation so you know what [field lookups](https://docs.djangoproject.com/en/dev/ref/models/querysets/#field-lookups) you can do. There are a lot of possibilities and you can only get more familiar with it through practice.

### Getting Specific Objects

Most of the time, we'll only need one. Let's do that and retrieve just one row from the database.

```
person = Person.objects.get(id=1)
```

The `.get()` method does __not__ return a `QuerySet`, it only returns an instance.

The `id` field is automatically added by Django if we didn't set `primary_key` in a field because each model is required to have exactly one primary key.

```
person = Person.objects.get(id__in=[1, 2])
```

If objects with `id`s of 1 and 2 exist, this will return an error because if `.get()` returns more than one result, it instead returns an error. Field lookups also work in `.get()` calls.

### Deleting

Let's delete a record.

```
person = Person.objects.get(id=1)
person.delete()
```

That's it! There's `.save()`, there's also `.delete()`.

### [Relationships](https://docs.djangoproject.com/en/dev/ref/models/fields/#module-django.db.models.fields.related)

Our coverage of models will be incomplete without discussing relationships. Django has several fields that represent them. Let's look at the various relationships that can be created.

#### ForeignKey

This field represents a many-to-one relationship. Relationships can be created by passing the model name as the argument to a relationship field like ForeignKey. The name can be the model class name itself, or the class name as a string. Strings will work for models that haven't been defined yet. For an object to create a recursive relationship with itself, the string 'self' is used.

Let's imagine that in our app, our users can be friends with each other so one will have relationship with many other profiles. Let's do that.

```
from django.contrib.auth.models import User
from django.db import models

class Profile(models.Model):
    name = models.CharField(max_length=42)
    photo = models.ImageField()
    created = models.DateField()
    friends = models.ForeignKey('self', on_delete=models.CASCADE)
```

The [`on_delete`](https://docs.djangoproject.com/en/dev/ref/models/fields/#django.db.models.ForeignKey.on_delete) argument tells Django what to do when the referenced object is deleted. Check the [documentation](https://docs.djangoproject.com/en/dev/ref/models/fields/#django.db.models.ForeignKey.on_delete) for details.

#### OneToOneField

We'll create a one-to-one relationship between our Profile model and the Django `User` model. We can import Django's built-in `User` model and create a one-to-one relationship with it. This is conceptually similar to `ForeignKey` with `unique=True`.

```
from django.contrib.auth.models import User
from django.db import models

class Profile(models.Model):
    name = models.CharField(max_length=42)
    photo = models.ImageField()
    created = models.DateField()
    user = models.OneToOneField(User)
```

#### Accessing Related Models

Each `Profile` instance will have an exclusive relationship with a `User`. The `User` model can be accessed from `Profile`.

```
profile = Profile.objects.get(id=1)

user = profile.user
user.id
```

The reverse is also possible. A related `Profile` can be accessed from `User` using its `related_name`. The default `related_name` is the name of the class (in lower case), but it can be changed. Field lookups can also be used on related models.

```
user = User.objects.get(profile__id=1)

profile.user == user
user.profile == profile
```

Here we add a `related_name` to the `models.OneToOneField()` and access the related model using it.

```
# model with OneToOneField updated with related_name
class Profile(models.Model):
    name = models.CharField(max_length=42)
    photo = models.ImageField()
    created = models.DateField()
    user = models.OneToOneField(User, related_name='details')
```

Here we retrieve a `Profile` object that matches our `User` by passing an instance.

```
profile = Profile.objects.get(user=user)

profile.user == user
user.details == profile
```

#### Django User Model

Referring directly to the `User` model won't work in projects or apps that use a custom `User` model. Another way to refer to the `User` is by using the `AUTH_USER_MODEL` setting. This way, when a custom `User` is used, your related model will still refer to the right model. This is the recommended way to add information about your users and create a relationship with the `User` model.

```
from django.conf import settings
from django.db import models

class Profile(models.Model):
    name = models.CharField(max_length=42)
    photo = models.ImageField(null=True, blank=True)
    created = models.DateField(default=datetime.now)
    user = models.OneToOneField(settings.AUTH_USER_MODEL)
```

Another way to retrieve the `User` model is by using `django.contrib.auth.get_user_model()`. However, it can't guarantee that the models are already loaded. For models, `settings.AUTH_USER_MODEL` is recommended while `get_user_model()` may be used within controllers.

#### ManyToManyField

This time we'll create a model having a many-to-many relationship. In our service, search terms sent to our API are recorded. These search terms can be entered by different users so our `Search` model can be related to multiple users while users can also search for as many things as they want. The ManyToManyField is perfect for this.

```
from django.conf import settings
from django.db import models

class Search(models.Model):
    terms = models.CharField(max_length=80)
    user = models.ManyToManyField(settings.AUTH_USER_MODEL)
```